In [ ]:
#importing required libraries
import openai
import langchain
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import pinecone
from langchain.llms import openai

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import os

In [ ]:
##Reading Document
def read_Doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

In [ ]:
doc=read_Doc('Documents/')
len(doc)

In [ ]:
##Divide the docs into chunks

def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap) 
    doc=text_splitter.split_documents(docs)
    return docs   

In [ ]:
Documents=chunk_data(docs=doc)
len(Documents)

In [ ]:
##Embadding Technique of OPENAI

import os
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
load_dotenv(Path(".env"))
#print(os.getenv('OPENAI_API_KEY'))
embeddings = OpenAIEmbeddings(api_key=os.getenv('OPENAI_API_KEY'))
embeddings

In [ ]:
vectors = embeddings.embed_query('How are you?')
len(vectors)

In [ ]:
#Vector search db in pinecone
import pinecone
pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'),
    environment=os.getenv('PINECONE_ENVIRONMENT')
)
index_name =os.getenv('PINECONE_INDEX_NAME')



In [ ]:
from langchain.vectorstores import Pinecone
index=Pinecone.from_documents(doc,embeddings,index_name=index_name)

In [ ]:
##Cosine Similarity Retrieve Results from VectorDB

def retrieve_query(query,k=2):
    matching_results=index.similarity_search(query,k=k)
    return matching_results

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI



In [ ]:
llm=OpenAI(model_name="gpt-3.5-turbo-instruct",temperature=0.5)
chain=load_qa_chain(llm,chain_type="stuff")

In [ ]:
##sSearch answers from VectorDB

def retireve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response
    

In [ ]:
q="How is technology being used to streamline hospital operations,improve the quality of patient care, and provide better outcomes for patients?"
answer = retireve_answers(q)
print(answer)

In [ ]:
index.delete(delete_all=True)